In [1]:
import json
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

sparql.setQuery("""
SELECT ?property ?propertyLabel ?propertyDescription ?altLabel ?type 
    WHERE {
    ?property a wikibase:Property .
    MINUS { ?property wdt:P31 wd:Q18644427 }
    ?property wikibase:propertyType ?type .
    OPTIONAL { ?property skos:altLabel ?altLabel . FILTER (lang(?altLabel) = "id") }
    SERVICE wikibase:label { bd:serviceParam wikibase:language "id" }   
    } ORDER BY ASC(xsd:integer(strafter(str(?property), concat(str(wd:), "P"))))
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [3]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

sparql.setQuery("""
SELECT ?property ?propertyLabel ?propertyDescription ?altLabel
    WHERE {
    ?property a wikibase:Property .
    MINUS { ?property wdt:P31 wd:Q18644427 }
    OPTIONAL { ?property skos:altLabel ?altLabel . FILTER (lang(?altLabel) = "en") }
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }   
    } ORDER BY ASC(xsd:integer(strafter(str(?property), concat(str(wd:), "P"))))
""")
sparql.setReturnFormat(JSON)
results2 = sparql.query().convert()

In [4]:
results

{'head': {'vars': ['property',
   'propertyLabel',
   'propertyDescription',
   'altLabel',
   'type']},
 'results': {'bindings': [{'property': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/P6'},
    'altLabel': {'xml:lang': 'id', 'type': 'literal', 'value': 'bupati'},
    'type': {'type': 'uri', 'value': 'http://wikiba.se/ontology#WikibaseItem'},
    'propertyLabel': {'xml:lang': 'id',
     'type': 'literal',
     'value': 'kepala pemerintahan'},
    'propertyDescription': {'xml:lang': 'id',
     'type': 'literal',
     'value': 'kepala pemerintahan kota, kabupaten, negara bagian, provinsi, negara atau lembaga pemerintahan lainnya'}},
   {'property': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/P6'},
    'altLabel': {'xml:lang': 'id',
     'type': 'literal',
     'value': 'perdana menteri'},
    'type': {'type': 'uri', 'value': 'http://wikiba.se/ontology#WikibaseItem'},
    'propertyLabel': {'xml:lang': 'id',
     'type': 'literal',
     'value': 'kepala pe

In [9]:
prop_list = {}
id_stack = []
alt_list = []
dt = ''
label = ''
desc = ''
for elem in results['results']['bindings']:
    idp = elem['property']['value'].split('/')[4]
    if(idp not in id_stack):
        if(len(id_stack) == 0):
            id_stack.append(idp)
            dt = elem['type']['value'].split('#')[1]
            label = elem['propertyLabel']['value']
            if('propertyDescription' in elem):
                desc = elem['propertyDescription']['value']
        else:
            alt_list.append(label)
            prop = {}
            prop['labelId'] = label
            prop['id'] = id_stack.pop()
            prop['aliasId'] = alt_list
            prop['descriptionId'] = desc
            prop['data type'] = dt
            if(dt == 'Monolingualtext'):
                dt = 'String'
            
            prop_list[prop['id']] = prop
            alt_list = []
            desc = ""
            
            id_stack.append(idp)
            dt = elem['type']['value'].split('#')[1]
            label = elem['propertyLabel']['value']
            if('propertyDescription' in elem):
                desc = elem['propertyDescription']['value']
            
    else:
        alt_list.append(elem['altLabel']['value'])

In [10]:
id_stack = []
alt_list = []
dt = ''
label = ''
desc = ''
for elem in results2['results']['bindings']:
    idp = elem['property']['value'].split('/')[4]
    if(idp not in id_stack):
        if(len(id_stack) == 0):
            id_stack.append(idp)
            label = elem['propertyLabel']['value']
            if('propertyDescription' in elem):
                desc = elem['propertyDescription']['value']
        else:
            ids = id_stack.pop()
            alt_list.append(label)
            prop_list[ids]['labelEn'] = label
            prop_list[ids]['aliasEn'] = alt_list
            prop_list[ids]['descriptionEn'] = desc
            
            alt_list = []
            desc = ""
            
            id_stack.append(idp)
            label = elem['propertyLabel']['value']
            if('propertyDescription' in elem):
                desc = elem['propertyDescription']['value']
            
    else:
        alt_list.append(elem['altLabel']['value'])

In [11]:
list(prop_list.values())

[{'labelId': 'kepala pemerintahan',
  'id': 'P6',
  'aliasId': ['perdana menteri',
   'presiden',
   'walikota',
   'kepala pemerintahan'],
  'descriptionId': 'kepala pemerintahan kota, kabupaten, negara bagian, provinsi, negara atau lembaga pemerintahan lainnya',
  'data type': 'WikibaseItem',
  'labelEn': 'head of government',
  'aliasEn': ['government headed by',
   'head of national government',
   'mayor',
   'prime minister',
   'governor',
   'premier',
   'first minister',
   'chancellor',
   'head of government'],
  'descriptionEn': 'head of the executive power of this town, city, municipality, state, country, or other governmental body'},
 {'labelId': 'video',
  'id': 'P10',
  'aliasId': ['video'],
  'descriptionId': 'video berkaitan',
  'data type': 'CommonsMedia',
  'labelEn': 'video',
  'aliasEn': ['media', 'gif', 'video'],
  'descriptionEn': 'relevant video. For images, use the property P18'},
 {'labelId': 'papan jalan bebas hambatan',
  'id': 'P14',
  'aliasId': ['papan 

In [8]:
with open('data/dump/property.json', 'w') as outfile:
    json.dump(list(prop_list.values()), outfile)